In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [6]:
base_url = "https://www.creditkarma.com/credit-cards/search-cc"
response = requests.get(base_url)

soup = BeautifulSoup(response.content, 'html.parser')
cards = soup.find(id="anchor")



In [7]:
credit_cards = cards.find_all("div", class_="jsx-2570701536")

for card in credit_cards:
    card_name = card.find("h2", class_="lh-copy mv0 f4 f3-ns")

    details = card.find("li", class_="flex flex-column bb b--light-gray pv2 ph3 w-50")
    if details:
        details_text = details.text.strip() 

        if "Cash back" in details_text or "Miles per dollar" in details_text:
            cashback_match = re.search(r'(\d+(?:\.\d+)?%)\s*[-–]\s*(\d+(?:\.\d+)?%)|(\d+(?:\.\d+)?%)', details_text)
            if cashback_match:
                if cashback_match.group(1) and cashback_match.group(2):
                    details_re = f"Cash back: {cashback_match.group(1)} - {cashback_match.group(2)}"
                elif cashback_match.group(3):
                    details_re = f"Cash back: {cashback_match.group(3)}"
            else:
                details_re = "No cash back"
        elif "APR" in details_text:
            apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*-\s*(\d+(?:\.\d+)?%))?', details_text)
            if apr_match:
                details_re = "APR: " + apr_match.group(1)
            else:
                details_re = "No APR"
        else:
            details_re = "No APR or cash back"
    

        annual_fee = card.find("li", class_="flex flex-column bb b--light-gray pv2 ph3 w-50 bl")
        annual_fee_text = annual_fee.text.strip() if annual_fee else "$0"
        annual_fee = re.search(r'\$\d+', annual_fee_text)
        if annual_fee:
            annual_fee = annual_fee.group()
        else:
            annual_fee = "$0"

        welcome_bonus = "$0"
    else:
        details = card.find("li", class_="flex flex-column bb b--light-gray pv2 ph3 ckoffer-w-third")
        details_text = details.text.strip() if details else "$0"

        if "Cash back" in details_text or "Miles per dollar" in details_text:
            cashback_match = re.search(r'(\d+(?:\.\d+)?%)\s*[-–]\s*(\d+(?:\.\d+)?%)|(\d+(?:\.\d+)?%)', details_text)
            if cashback_match:
                if cashback_match.group(1) and cashback_match.group(2):
                    details_re = f"Cash back: {cashback_match.group(1)} - {cashback_match.group(2)}"
                elif cashback_match.group(3):
                    details_re = f"Cash back: {cashback_match.group(3)}"
            else:
                details_re = "No cash back"
        elif "APR" in details_text:
            apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*-\s*(\d+(?:\.\d+)?%))?', details_text)
            if apr_match:
                details_re = "APR: " + apr_match.group(1)
            else:
                details_re = "No APR"
        else:
            details_re = "No APR or cash back"
        
        annual_fee = "$0"
        welcome_bonus = "$0"

        annual_or_bonus = card.find_all("li", class_="flex flex-column bb b--light-gray pv2 ph3 ckoffer-w-third bl")
        for item in annual_or_bonus:
            item_text = item.text.strip()

            if "Annual Fee" in item_text:
                annual_fee_match = re.search(r'\$\d+', item_text)
                if annual_fee_match:
                    annual_fee = annual_fee_match.group()

            if "Welcome" in item_text or "Bonus" in item_text or "Intro offer" in item_text:
                bonus_match = re.search(r'\$\d+[,\d]*', item_text) 
                if bonus_match:
                    welcome_bonus = bonus_match.group()

            
    
    print("Name:" ,card_name.text.strip())
    print(details_re)
    print("Annual Fee:" ,annual_fee) 
    print("Welcome Bonus:" ,welcome_bonus)
    print("")
    
   
   

Name: Indigo® Mastercard® Cashback Rewards
Cash back: 1.5%
Annual Fee: $75
Welcome Bonus: $0

Name: Milestone® Mastercard®
APR: 35.90%
Annual Fee: $75
Welcome Bonus: $0

Name: OpenSky® Launch Secured Visa® Credit Card
No APR or cash back
Annual Fee: $0
Welcome Bonus: $0

Name: Capital One Quicksilver Secured Cash Rewards Credit Card
No APR or cash back
Annual Fee: $0
Welcome Bonus: $0

Name: Wells Fargo Reflect® Card
No APR or cash back
Annual Fee: $0
Welcome Bonus: $0

Name: Upgrade Cash Rewards Visa®
Cash back: 1.5%
Annual Fee: $0
Welcome Bonus: $0

Name: Credit One Bank® Platinum Rewards Visa®
Cash back: 1%
Annual Fee: $0
Welcome Bonus: $0

Name: Prosper® Card
APR: 23.24%
Annual Fee: $0
Welcome Bonus: $0

Name: Credit One Bank American Express® Unlimited Rewards Card
Cash back: 1%
Annual Fee: $0
Welcome Bonus: $0

Name: Wells Fargo Autograph® Card
No APR or cash back
Annual Fee: $0
Welcome Bonus: $0

Name: Credit One Bank® Platinum Visa® for Rebuilding Credit
Cash back: 1%
Annual Fe

In [8]:
credit_cards = cards.find_all("div", class_="jsx-2570701536")

for card in credit_cards:
    card_name_tag = card.find("h2", class_="lh-copy mv0 f4 f3-ns")
    card_name = card_name_tag.text.strip() if card_name_tag else "No name"

    cashback = "No cash back"
    apr = "No APR"
    annual_fee = "$0"
    welcome_bonus = "$0"
    cashback = "No cash back"
    points = "No points"

    # Get all info boxes
    info_boxes = card.find_all("li")
    
    for item in info_boxes[:4]:
        text = item.text.strip()

        # Cash back or Miles
        if "Cash back" in text or "Miles per dollar" in text or "Rewards rate" in text:
            rewards_match = re.search(
            r'(\d+(?:\.\d+)?[x%])\s*[-–]\s*(\d+(?:\.\d+)?[x%])|(\d+(?:\.\d+)?[x%])', text, re.IGNORECASE)

        if rewards_match:
            if rewards_match.group(1) and rewards_match.group(2):
                first_unit = rewards_match.group(1)[-1]
                if first_unit == '%':
                    cashback = f"Cash back: {rewards_match.group(1)} - {rewards_match.group(2)}"
                elif first_unit.lower() == 'x':
                    points = f"Points: {rewards_match.group(1)} - {rewards_match.group(2)}"
            elif rewards_match.group(3):
                single_unit = rewards_match.group(3)[-1]
                if single_unit == '%':
                    cashback = f"Cash back: {rewards_match.group(3)}"
                elif single_unit.lower() == 'x':
                    points = f"Points: {rewards_match.group(3)}"

        # APR
        if "APR" in text:
            apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*[-–]\s*(\d+(?:\.\d+)?%))?', text)
            if apr_match:
                if apr_match.group(2):
                    apr = f"APR: {apr_match.group(1)} - {apr_match.group(2)}"
                else:
                    apr = f"APR: {apr_match.group(1)}"

        # Annual Fee
        if "Annual fee" in text:
            fee_match = re.search(r'\$\d+', text)
            if fee_match:
                annual_fee = fee_match.group()
            else:
                annual_fee = "$0"

        # Welcome Bonus
        if "Welcome bonus" in text or "Intro offer" in text or "Bonus" in text:
            bonus_match = re.search(r'(\$\d{2,}[,\d]*)|(\d{1,3}(?:,\d{3})*)\s*(Points|Miles)', text)
            if bonus_match:
                if bonus_match.group(1):  # Dollar bonus
                    welcome_bonus = f"{bonus_match.group(1)}"
                elif bonus_match.group(2) and bonus_match.group(3):  # Points or miles
                    amount = bonus_match.group(2)
                    unit = bonus_match.group(3).capitalize()
                    welcome_bonus = f"{amount} {unit}"
    #Print output
    print("Name:", card_name)
    if cashback != "No cash back":
        print(cashback)
    if points != "No points":
        print(points + " Miles per dollar")
    print("Annual Fee:", annual_fee)
    print(f"Welcome Bonus: {welcome_bonus}")
    print("")
    
    


Name: Indigo® Mastercard® Cashback Rewards
Cash back: 1.5%
Annual Fee: $75
Welcome Bonus: $0

Name: Milestone® Mastercard®
Cash back: 1.5%
Annual Fee: $75
Welcome Bonus: $0

Name: OpenSky® Launch Secured Visa® Credit Card
Cash back: 10%
Annual Fee: $24
Welcome Bonus: $0

Name: Capital One Quicksilver Secured Cash Rewards Credit Card
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $0

Name: Wells Fargo Reflect® Card
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $0

Name: Upgrade Cash Rewards Visa®
Cash back: 1.5%
Annual Fee: $0
Welcome Bonus: $0

Name: Credit One Bank® Platinum Rewards Visa®
Cash back: 1%
Annual Fee: $0
Welcome Bonus: $0

Name: Prosper® Card
Cash back: 1%
Annual Fee: $0
Welcome Bonus: $0

Name: Credit One Bank American Express® Unlimited Rewards Card
Cash back: 1%
Annual Fee: $0
Welcome Bonus: $0

Name: Wells Fargo Autograph® Card
Points: 1x - 3x Miles per dollar
Annual Fee: $0
Welcome Bonus: 20,000 Points

Name: Credit One Bank® Platinum Visa® for Rebuilding Cr

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

headers = {"User-Agent": "Mozilla/5.0"}

# URLs by credit score tier
pages = {
    "Excellent": "https://www.creditkarma.com/credit-cards/excellent-credit",
    "Good": "https://www.creditkarma.com/credit-cards/good-credit",
    "Fair": "https://www.creditkarma.com/credit-cards/fair-credit",
    "Bad": "https://www.creditkarma.com/credit-cards/bad-credit",
}

all_cards = []

# Loop through each page
for credit_score_label, url in pages.items():
    print(f"Scraping cards for: {credit_score_label} credit...")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    credit_cards = soup.find_all("div", class_="jsx-2570701536") 

    for card in credit_cards[:10]:
        card_name_tag = card.find("h2", class_="lh-copy mv0 f4 f3-ns")
        card_name = card_name_tag.text.strip() if card_name_tag else "No name"

        cashback = "No cash back"
        points = "No points"
        apr = "No APR"
        annual_fee = "$0"
        welcome_bonus = "No welcome bonus"

        info_boxes = card.find_all("li")
        for item in info_boxes[:4]:
            text = item.text.strip()

            # Rewards
            if "Cash back" in text or "Miles per dollar" in text or "Rewards rate" in text:
                rewards_match = re.search(
                    r'(\d+(?:\.\d+)?[x%])\s*[-–]\s*(\d+(?:\.\d+)?[x%])|(\d+(?:\.\d+)?[x%])',
                    text, re.IGNORECASE
                )
                if rewards_match:
                    if rewards_match.group(1) and rewards_match.group(2):
                        unit = rewards_match.group(1)[-1]
                        if unit == '%':
                            cashback = f"{rewards_match.group(1)} - {rewards_match.group(2)}"
                        elif unit.lower() == 'x':
                            points = f"{rewards_match.group(1)} - {rewards_match.group(2)}"
                    elif rewards_match.group(3):
                        unit = rewards_match.group(3)[-1]
                        if unit == '%':
                            cashback = f"{rewards_match.group(3)}"
                        elif unit.lower() == 'x':
                            points = f"{rewards_match.group(3)}"

            # APR
            if "APR" in text:
                apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*[-–]\s*(\d+(?:\.\d+)?%))?', text)
                if apr_match:
                    if apr_match.group(2):
                        apr = f"{apr_match.group(1)} - {apr_match.group(2)}"
                    else:
                        apr = f"{apr_match.group(1)}"

            # Annual Fee
            if "Annual fee" in text.lower():
                fee_match = re.search(r'\$\d+', text)
                if fee_match:
                    annual_fee = fee_match.group()

            # Welcome Bonus
            if "welcome bonus" in text.lower() or "intro offer" in text.lower() or "bonus" in text.lower():
                bonus_match = re.search(r'(\$\d{2,}[,\d]*)|(\d{1,3}(?:,\d{3})*)\s*(Points|Miles)', text)
                if bonus_match:
                    if bonus_match.group(1):  # Dollar bonus
                        welcome_bonus = bonus_match.group(1)
                    elif bonus_match.group(2) and bonus_match.group(3):
                        welcome_bonus = f"{bonus_match.group(2)} {bonus_match.group(3).capitalize()}"

        # Store result
        all_cards.append({
            "Name": card_name,
            "Credit Score Tier": credit_score_label,
            "Cashback": cashback,
            "Points": points,
            "APR": apr,
            "Annual Fee": annual_fee,
            "Welcome Bonus": welcome_bonus
        })

# Convert to DataFrame and Save
df = pd.DataFrame(all_cards)
print(df)
#print(all_cards)
df.to_csv("labeled_credit_cards.csv", index=False)



Scraping cards for: Excellent credit...
Scraping cards for: Good credit...
Scraping cards for: Fair credit...
Scraping cards for: Bad credit...
                                                 Name Credit Score Tier  \
0                      Chase Sapphire Preferred® Card         Excellent   
1    Capital One Quicksilver Cash Rewards Credit Card         Excellent   
2                               Citi Simplicity® Card         Excellent   
3             Capital One Venture Rewards Credit Card         Excellent   
4                          Upgrade Cash Rewards Visa®         Excellent   
5         Bank of America® Travel Rewards credit card         Excellent   
6                              Citi Double Cash® Card         Excellent   
7   Bank of America® Customized Cash Rewards credi...         Excellent   
8                       Citi® Diamond Preferred® Card         Excellent   
9          Capital One Savor Cash Rewards Credit Card         Excellent   
10                         Upgr

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

headers = {"User-Agent": "Mozilla/5.0"}

# URLs by credit score tier
pages = {
    "Excellent": "https://www.creditkarma.com/credit-cards/excellent-credit",
    "Good": "https://www.creditkarma.com/credit-cards/good-credit",
    "Fair": "https://www.creditkarma.com/credit-cards/fair-credit",
    "Bad": "https://www.creditkarma.com/credit-cards/bad-credit",
}

all_cards = []

# Loop through each page
for credit_score_label, url in pages.items():
    print(f"Scraping cards for: {credit_score_label} credit...")
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    credit_cards = soup.find_all("div", class_="jsx-2570701536") 

    for card in credit_cards[:10]:
        card_name_tag = card.find("h2", class_="lh-copy mv0 f4 f3-ns")
        card_name = card_name_tag.text.strip() if card_name_tag else "No name"

        cashback = "No cash back"
        points = "No points"
        apr = "No APR"
        annual_fee = "$0"
        welcome_bonus = "No welcome bonus"

        info_boxes = card.find_all("li")
        for item in info_boxes[:4]:
            text = item.text.strip()

            # Rewards
            if "Cash back" in text or "Miles per dollar" in text or "Rewards rate" in text:
                rewards_match = re.search(
                    r'(\d+(?:\.\d+)?[x%])\s*[-–]\s*(\d+(?:\.\d+)?[x%])|(\d+(?:\.\d+)?[x%])',
                    text, re.IGNORECASE
                )
                if rewards_match:
                    if rewards_match.group(1) and rewards_match.group(2):
                        unit = rewards_match.group(1)[-1]
                        if unit == '%':
                            cashback = f"Cash back: {rewards_match.group(1)} - {rewards_match.group(2)}"
                        elif unit.lower() == 'x':
                            points = f"Points: {rewards_match.group(1)} - {rewards_match.group(2)}"
                    elif rewards_match.group(3):
                        unit = rewards_match.group(3)[-1]
                        if unit == '%':
                            cashback = f"Cash back: {rewards_match.group(3)}"
                        elif unit.lower() == 'x':
                            points = f"Points: {rewards_match.group(3)}"

            # APR
            if "APR" in text:
                apr_match = re.search(r'(\d+(?:\.\d+)?%)(?:\s*[-–]\s*(\d+(?:\.\d+)?%))?', text)
                if apr_match:
                    if apr_match.group(2):
                        apr = f"APR: {apr_match.group(1)} - {apr_match.group(2)}"
                    else:
                        apr = f"APR: {apr_match.group(1)}"

            # Annual Fee
            if "Annual fee" in text.lower():
                fee_match = re.search(r'\$\d+', text)
                if fee_match:
                    annual_fee = fee_match.group()

            # Welcome Bonus
            if "welcome bonus" in text.lower() or "intro offer" in text.lower() or "bonus" in text.lower():
                bonus_match = re.search(r'(\$\d{2,}[,\d]*)|(\d{1,3}(?:,\d{3})*)\s*(Points|Miles)', text)
                if bonus_match:
                    if bonus_match.group(1):  # Dollar bonus
                        welcome_bonus = bonus_match.group(1)
                    elif bonus_match.group(2) and bonus_match.group(3):
                        welcome_bonus = f"{bonus_match.group(2)} {bonus_match.group(3).capitalize()}"

       #Print output
        print("Name:", card_name)
        if cashback != "No cash back":
            print(cashback)
        if points != "No points":
            print(points + " Miles per dollar")
        print("Annual Fee:", annual_fee)
        print(f"Welcome Bonus: {welcome_bonus}")
        print("")


Scraping cards for: Excellent credit...
Name: Upgrade Cash Rewards Visa®
Cash back: 1.5%
Annual Fee: $0
Welcome Bonus: No welcome bonus

Name: Capital One Venture Rewards Credit Card
Points: 2x - 5x Miles per dollar
Annual Fee: $0
Welcome Bonus: $915

Name: Chase Sapphire Preferred® Card
Points: 1x - 5x Miles per dollar
Annual Fee: $0
Welcome Bonus: 100,000 Points

Name: Capital One Quicksilver Cash Rewards Credit Card
Cash back: 1.5% - 5%
Annual Fee: $0
Welcome Bonus: $200

Name: Citi Simplicity® Card
Annual Fee: $0
Welcome Bonus: No welcome bonus

Name: Citi Double Cash® Card
Cash back: 1% - 5%
Annual Fee: $0
Welcome Bonus: $200

Name: Citi® Diamond Preferred® Card
Annual Fee: $0
Welcome Bonus: No welcome bonus

Name: Bank of America® Travel Rewards credit card
Points: 1.5x - 3x Miles per dollar
Annual Fee: $0
Welcome Bonus: $250

Name: Bank of America® Customized Cash Rewards credit card
Cash back: 1% - 3%
Annual Fee: $0
Welcome Bonus: $200

Name: Capital One Savor Cash Rewards Cred

In [11]:
dataset = pd.read_csv("labeled_credit_cards.csv")

In [12]:
print(dataset.head())

                                               Name Credit Score Tier  \
0                    Chase Sapphire Preferred® Card         Excellent   
1  Capital One Quicksilver Cash Rewards Credit Card         Excellent   
2                             Citi Simplicity® Card         Excellent   
3           Capital One Venture Rewards Credit Card         Excellent   
4                        Upgrade Cash Rewards Visa®         Excellent   

       Cashback     Points              APR Annual Fee     Welcome Bonus  
0  No cash back    1x - 5x           No APR         $0    100,000 Points  
1     1.5% - 5%  No points               0%         $0              $200  
2  No cash back  No points  18.24% - 28.99%         $0  No welcome bonus  
3  No cash back    2x - 5x           No APR         $0              $915  
4          1.5%  No points           No APR         $0  No welcome bonus  


In [13]:
dataset_clean = dataset.copy()
dataset_clean = pd.get_dummies(dataset_clean, columns=["Credit Score Tier"])
dataset_clean



,Name,Cashback,Points,APR,Annual Fee,Welcome Bonus,Credit Score Tier_Bad,Credit Score Tier_Excellent,Credit Score Tier_Fair,Credit Score Tier_Good
0,Chase Sapphire Preferred® Card,No cash back,1x - 5x,No APR,$0,"100,000 Points",False,True,False,False
1,Capital One Quicksilver Cash Rewards Credit Card,1.5% - 5%,No points,0%,$0,$200,False,True,False,False
2,Citi Simplicity® Card,No cash back,No points,18.24% - 28.99%,$0,No welcome bonus,False,True,False,False
3,Capital One Venture Rewards Credit Card,No cash back,2x - 5x,No APR,$0,$915,False,True,False,False
4,Upgrade Cash Rewards Visa®,1.5%,No points,No APR,$0,No welcome bonus,False,True,False,False
5,Bank of America® Travel Rewards credit card,No cash back,1.5x - 3x,No APR,$0,$250,False,True,False,False
6,Citi Double Cash® Card,1% - 5%,No points,No APR,$0,$200,False,True,False,False
7,Bank of America® Customized Cash Rewards credi...,1% - 3%,No points,No APR,$0,$200,False,True,False,False
8,Citi® Diamond Preferred® Card,No cash back,No points,17.24% - 27.99%,$0,No welcome bonus,False,True,False,False
9,Capital One Savor Cash Rewards Credit Card,1% - 8%,No points,0%,$0,$200,False,True,False,False


In [14]:
def extract_cashback(cashback_str):
    if isinstance(cashback_str, str) and "%" in cashback_str:
        matches = re.findall(r'(\d+(?:\.\d+)?)%', cashback_str)
        if matches:
            return float(max(matches))
    return 0.0

dataset_clean["Cashback"] = dataset_clean["Cashback"].apply(extract_cashback)

In [15]:
def extract_points(points_str):
    if isinstance(points_str, str) and "x" in points_str:
        matches = re.findall(r'(\d+(?:\.\d+)?)x', points_str)
        if matches:
            return float(max(matches))
    return 0.0

dataset_clean["Points"] = dataset_clean["Points"].apply(extract_points)

In [16]:
dataset_clean["Annual Fee"] = dataset_clean["Annual Fee"].apply(lambda x: int(re.sub(r'[^\d]', '', x)) if "$" in str(x) else 0)

In [17]:
def extract_apr(apr_str):
    if isinstance(apr_str, str):
        matches = re.findall(r'(\d+(?:\.\d+)?)%', apr_str)
        if matches:
            nums = [float(num) for num in matches]
            return sum(nums) / len(nums)
    return 0.0

dataset_clean["APR"] = dataset_clean["APR"].apply(extract_apr)

In [18]:
def extract_welcome_bonus(welcome_bonus_str):
    if isinstance(welcome_bonus_str, str):
        if "$" in welcome_bonus_str:
            return int(re.sub(r'[^\d]', '', welcome_bonus_str))
        elif "Points" in welcome_bonus_str or "Miles" in welcome_bonus_str:
            return int(re.sub(r'[^\d]', '', welcome_bonus_str)) // 100
    return 0.0

dataset_clean["Welcome Bonus"] = dataset_clean["Welcome Bonus"].apply(extract_welcome_bonus)

In [19]:
dataset_clean

,Name,Cashback,Points,APR,Annual Fee,Welcome Bonus,Credit Score Tier_Bad,Credit Score Tier_Excellent,Credit Score Tier_Fair,Credit Score Tier_Good
0,Chase Sapphire Preferred® Card,0.0,5.0,0.000,0,1000.0,False,True,False,False
1,Capital One Quicksilver Cash Rewards Credit Card,5.0,0.0,0.000,0,200.0,False,True,False,False
2,Citi Simplicity® Card,0.0,0.0,23.615,0,0.0,False,True,False,False
3,Capital One Venture Rewards Credit Card,0.0,5.0,0.000,0,915.0,False,True,False,False
4,Upgrade Cash Rewards Visa®,1.5,0.0,0.000,0,0.0,False,True,False,False
5,Bank of America® Travel Rewards credit card,0.0,3.0,0.000,0,250.0,False,True,False,False
6,Citi Double Cash® Card,5.0,0.0,0.000,0,200.0,False,True,False,False
7,Bank of America® Customized Cash Rewards credi...,3.0,0.0,0.000,0,200.0,False,True,False,False
8,Citi® Diamond Preferred® Card,0.0,0.0,22.615,0,0.0,False,True,False,False
9,Capital One Savor Cash Rewards Credit Card,8.0,0.0,0.000,0,200.0,False,True,False,False


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

feature_cols = ["Cashback", "Points", "APR", "Annual Fee", "Welcome Bonus", "Credit Score Tier_Good", "Credit Score Tier_Fair", "Credit Score Tier_Bad"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset_clean[feature_cols])

knn = NearestNeighbors(n_neighbors=5, algorithm='auto')
knn.fit(X_scaled)

def recommend_similar_cards(index, top_n=5):
    distances, indices = knn.kneighbors([X_scaled[index]], n_neighbors=top_n)
    return dataset_clean.iloc[indices[0]][["Name", "Cashback", "Points", "APR", "Annual Fee", "Welcome Bonus"]]

recommendations = recommend_similar_cards(10)
recommendations.reset_index(drop=True, inplace=True)
recommendations

,Name,Cashback,Points,APR,Annual Fee,Welcome Bonus
0,Upgrade Cash Rewards Visa®,1.5,0.0,0.0,0,0.0
1,Capital One Quicksilver Cash Rewards Credit Card,5.0,0.0,0.0,0,200.0
2,The New United Gateway℠ Card,0.0,2.0,0.0,0,426.0
3,Upgrade Cash Rewards Visa®,1.5,0.0,0.0,0,0.0
4,Capital One Savor Cash Rewards Credit Card,8.0,0.0,0.0,0,200.0
